In [18]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from helpers import load_pint_data, safe_parse_quantity
import pandas as pd
import pint
from tqdm import tqdm

### Import 

In [19]:
ROOT = Path().cwd().parent
PARAMETER_SPACE = "03"
DATA_TYPE = "Training"
ROOT

PosixPath('/Users/thomassimader/Documents/NIRB')

In [20]:
import sys
sys.path.append(str(ROOT))
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.entropy import calculate_S_therm

In [21]:
import_folder = ROOT / "Snapshots" / PARAMETER_SPACE 
import_folder.exists()

True

In [5]:
temperatures = np.load(import_folder      / "Exports" / f"{DATA_TYPE}_temperatures.npy" )
temperatures_diff = np.load(import_folder / "Exports" / f"{DATA_TYPE}_temperatures_diff.npy" )

## Plot

### Min max temperature difference

In [6]:
comsol_data  = COMSOL_VTU(import_folder / DATA_TYPE / f"{DATA_TYPE}_000.vtu")

In [7]:
N_SNAPS, N_TIMES, N_POINTS = temperatures_diff.shape

colors = px.colors.sample_colorscale("jet", [n/(N_SNAPS) for n in range(N_SNAPS)])

fig = go.Figure()
for idx_snap, temperature_array in enumerate(temperatures_diff):
# customdata=np.stack(([t_h] * len(comsol_data.times), [host_k] * len(comsol_data.times)) , axis=-1),
    input_args = {'x' : list(comsol_data.times.values()),
                #   'customdata' : customdata,
                #   'hovertemplate' : "Time: %{x:.2e}<br>Temperature: %{y:.2f}<br>T_h: %{customdata[0]:.2f}<br><br>host_k: %{customdata[1]:.2e}<br>",
                    'legendgroup' : f"{idx_snap:03d}",
                    'name' : f"{idx_snap:03d}",
                    'line' : dict(color=colors[idx_snap]),
                    'opacity': 0.4}
    fig.add_trace(go.Scatter(y=np.max(temperature_array, axis=1), **input_args))
    fig.add_trace(go.Scatter(y=np.min(temperature_array, axis=1), **input_args))

fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title_text=f'Temperature Difference (T-T0) - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}', # title of plot
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Temperature [K]"     # Optional: label for y-axis
)
fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_MinMaxTemperatures.png")
fig.show()


### Simulation Time

In [8]:
sim_times_path = import_folder / "Exports" / f"{DATA_TYPE}_sim_times.npy"
sim_times = np.load(sim_times_path)

In [9]:
fig = go.Figure(data=[go.Histogram(
    x=sim_times / 60,
    histnorm='percent',
    name='control', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        # start=-4.0,
        # end=3.0,
        size=2,
        ),
    marker_color='#EB89B5',
    opacity=0.75,
     texttemplate="%{x}"
    )])

fig.update_layout(
    title_text=f'Simulation Time - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}', # title of plot
    xaxis_title_text='Time [min]', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargroupgap=0.1, # gap between bars of the same location coordinates
)
fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_SimulationTime.png")
fig.show()


## Calculate Entropy


In [10]:
param_folder = import_folder / "Exports"
param_files = sorted([path for path in param_folder.rglob(f"{DATA_TYPE}*.csv")])
param_files

[PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_000_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_001_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_002_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_003_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_004_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_005_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_006_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_007_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_008_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/02/Exports/Training_009_parameters.csv'),


In [11]:
ureg = pint.UnitRegistry()
# ureg.define('@alias degC = degree_Celsius')
ureg.define('@alias degC = degree_Celsius')
# ureg.define("degree_Celsius = degC = degC")
# ureg.define('degree_Celsius = kelvin - 273.15')
"degree_Celsius" in ureg

True

In [12]:
df = pd.read_csv(param_files[0], index_col = 0)
df.columns

Index(['quantity'], dtype='object')

In [13]:
df['quantity_pint'] = df[df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
df

,quantity,quantity_pint
aspect_ratio_dip,1.0 dimensionless,1.0 dimensionless
aspect_ratio_strike,1.25 dimensionless,1.25 dimensionless
beta,0.00059 / kelvin,0.00059 / kelvin
dip,1.0471975511965976 radian,1.0471975511965976 radian
fault_Cp,800.0 meter ** 2 / kelvin / second ** 2,800.0 meter ** 2 / kelvin / second ** 2
fault_k_long,1.9498445997580416e-14 meter ** 2,1.9498445997580416e-14 meter ** 2
fault_k_trans,8.511380382023759e-15 meter ** 2,8.511380382023759e-15 meter ** 2
fault_lambda,2.0 kilogram * meter / kelvin / second ** 3,2.0 kilogram * meter / kelvin / second ** 3
fault_phi,0.3 dimensionless,0.3 dimensionless
fault_rho,2500.0 kilogram / meter ** 3,2500.0 kilogram / meter ** 3


In [14]:
lambda_therm = (1 - df.loc['host_phi', "quantity_pint"]).magnitude * df.loc['host_lambda', "quantity_pint"].magnitude + \
                df.loc['host_phi', "quantity_pint"].magnitude * (4.2)
T0 = 0.5 * (df.loc["T_h", "quantity_pint"] + (20 + 273.15) * ureg.kelvin)
delta_T = (df.loc['T_h', "quantity_pint"] - (20 + 273.15) * ureg.kelvin)
L = df.loc["H", "quantity_pint"]
print(f"{lambda_therm=}")
print(f"{T0=}")
print(f"{delta_T=}")
print(f"{L=}")

lambda_therm=2.0022
T0=<Quantity(356.475, 'kelvin')>
delta_T=<Quantity(126.65, 'kelvin')>
L=<Quantity(4000.0, 'meter')>


In [15]:
entropy_gen_unit = np.zeros((temperatures.shape[0], temperatures.shape[1]))
entropy_gen_number = np.zeros_like(entropy_gen_unit)
comsol_data.mesh.clear_point_data()

for idx_snap in range(temperatures.shape[0]):
    for idx_time in range(temperatures.shape[1]):
        comsol_data.mesh.point_data["temp_field"] = temperatures[idx_snap, idx_time, :]
        cell_mesh = comsol_data.mesh.point_data_to_cell_data()
        cell_mesh.cell_data.keys()
        temp_grad = cell_mesh.compute_derivative("temp_field").cell_data["gradient"]
        s0 = calculate_S_therm(lambda_therm,
                        T0.magnitude,
                        temp_grad)

        s0_total = np.sum(s0 * comsol_data.mesh.compute_cell_sizes()["Volume"])

        s0_characteristic = (lambda_therm * delta_T.magnitude**2) / (L.magnitude**2 * T0.magnitude**2)
        entropy_number = s0_total / s0_characteristic / comsol_data.mesh.volume 
        #
        entropy_gen_unit[idx_snap, idx_time] = s0_total
        entropy_gen_number[idx_snap, idx_time] = entropy_number
        
         



### Plot entropy generation

In [16]:
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_unit):
    fig.add_trace(go.Scatter(x=list(comsol_data.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [W/K]"     # Optional: label for y-axis
)

fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_EntropyGeneration.png")
fig.show()

### Plot entropy generation number

In [17]:
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_number):
    fig.add_trace(go.Scatter(x=list(comsol_data.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation number - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [-]"     # Optional: label for y-axis
)

fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_EntropyGenerationNumber.png")
fig.show()